In [1]:
%env OMP_NUM_THREADS = 4

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer, f1_score,accuracy_score, cohen_kappa_score, log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
data = pd.read_csv("OJ.csv")

In [4]:
data
data.info()
data.describe()

,Unnamed: 0,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1066,CH,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,Yes,0.000000,0.053763,0.23,0
1066,1067,CH,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1067,1068,MM,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1068,1069,CH,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,Yes,0.112676,0.000000,0.27,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1070 non-null   int64  
 1   Purchase        1070 non-null   object 
 2   WeekofPurchase  1070 non-null   int64  
 3   StoreID         1070 non-null   int64  
 4   PriceCH         1070 non-null   float64
 5   PriceMM         1070 non-null   float64
 6   DiscCH          1070 non-null   float64
 7   DiscMM          1070 non-null   float64
 8   SpecialCH       1070 non-null   int64  
 9   SpecialMM       1070 non-null   int64  
 10  LoyalCH         1070 non-null   float64
 11  SalePriceMM     1070 non-null   float64
 12  SalePriceCH     1070 non-null   float64
 13  PriceDiff       1070 non-null   float64
 14  Store7          1070 non-null   object 
 15  PctDiscMM       1070 non-null   float64
 16  PctDiscCH       1070 non-null   float64
 17  ListPriceDiff   1070 non-null   f

,Unnamed: 0,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
count,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000
mean,535.500000,254.381308,3.959813,1.867421,2.085411,0.051860,0.123364,0.147664,0.161682,0.565782,1.962047,1.815561,0.146486,0.059298,0.027314,0.217991,1.630841
std,309.026698,15.558286,2.308984,0.101970,0.134386,0.117474,0.213834,0.354932,0.368331,0.307843,0.252697,0.143384,0.271563,0.101760,0.062232,0.107535,1.430387
min,1.000000,227.000000,1.000000,1.690000,1.690000,0.000000,0.000000,0.000000,0.000000,0.000011,1.190000,1.390000,-0.670000,0.000000,0.000000,0.000000,0.000000
25%,268.250000,240.000000,2.000000,1.790000,1.990000,0.000000,0.000000,0.000000,0.000000,0.325257,1.690000,1.750000,0.000000,0.000000,0.000000,0.140000,0.000000
50%,535.500000,257.000000,3.000000,1.860000,2.090000,0.000000,0.000000,0.000000,0.000000,0.600000,2.090000,1.860000,0.230000,0.000000,0.000000,0.240000,2.000000
75%,802.750000,268.000000,7.000000,1.990000,2.180000,0.000000,0.230000,0.000000,0.000000,0.850873,2.130000,1.890000,0.320000,0.112676,0.000000,0.300000,3.000000
max,1070.000000,278.000000,7.000000,2.090000,2.290000,0.500000,0.800000,1.000000,1.000000,0.999947,2.290000,2.090000,0.640000,0.402010,0.252688,0.440000,4.000000


In [5]:
data = data.drop(['Unnamed: 0'],axis=1)

In [6]:
data.dtypes

Purchase           object
WeekofPurchase      int64
StoreID             int64
PriceCH           float64
PriceMM           float64
DiscCH            float64
DiscMM            float64
SpecialCH           int64
SpecialMM           int64
LoyalCH           float64
SalePriceMM       float64
SalePriceCH       float64
PriceDiff         float64
Store7             object
PctDiscMM         float64
PctDiscCH         float64
ListPriceDiff     float64
STORE               int64
dtype: object

In [7]:
data['Purchase'].value_counts()

CH    653
MM    417
Name: Purchase, dtype: int64

In [8]:
data['Purchase'] = data.apply(lambda X: '0' if X.Purchase == 'CH' else '1', axis=1)

In [9]:
data['Purchase'].value_counts()

0    653
1    417
Name: Purchase, dtype: int64

In [10]:
data

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,0,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,0,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,0,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,1,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,0,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,0,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,Yes,0.000000,0.053763,0.23,0
1066,0,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1067,1,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1068,0,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,Yes,0.112676,0.000000,0.27,0


In [11]:
#pandas_profiling.ProfileReport(data)

In [12]:
data.dtypes

Purchase           object
WeekofPurchase      int64
StoreID             int64
PriceCH           float64
PriceMM           float64
DiscCH            float64
DiscMM            float64
SpecialCH           int64
SpecialMM           int64
LoyalCH           float64
SalePriceMM       float64
SalePriceCH       float64
PriceDiff         float64
Store7             object
PctDiscMM         float64
PctDiscCH         float64
ListPriceDiff     float64
STORE               int64
dtype: object

In [13]:
data['STORE'] = data['STORE'].astype('category')
data['StoreID'] = data['StoreID'].astype('category')
data['SpecialCH'] = data['SpecialCH'].astype('category')
data['SpecialMM'] = data['SpecialMM'].astype('category')
data['Store7'] = data['Store7'].astype('category')

In [14]:
data = data.drop(['PctDiscMM','PctDiscCH'],axis=1)

In [15]:
data['Store7'] = data.apply(lambda X: '0' if X.Purchase == 'No' else '1', axis=1)

In [16]:
target_col = 'Purchase'

X = data.drop([target_col], axis=1)
y = data[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(856, 15)
(214, 15)
(856,)
(214,)


In [18]:
score_func = make_scorer(roc_auc_score, greater_is_better=True)

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier(random_state=77300)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,scoring = score_func, cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   50.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 22.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,...
                                              random_state=77300, verbose=0,
                                  

In [19]:
grid_search.best_params_
grid_search.best_estimator_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 100}

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=80, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=77300,
                       verbose=0, warm_start=False)

In [20]:
pred=grid_search.predict(X_test)
predprob = grid_search.predict_proba(X_test)[:,1]

In [21]:
confusion_matrix(y_test,pred)
roc_auc_score(y_test,predprob)
print(classification_report(y_test, pred))
print("Accuracy = {:.2f}".format(accuracy_score(y_test, pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, pred, pos_label='0')))
print("Log Loss = {:.2f}".format(log_loss(y_test, pred)))

array([[112,  18],
       [ 22,  62]], dtype=int64)

0.8685897435897436

              precision    recall  f1-score   support

           0       0.84      0.86      0.85       130
           1       0.78      0.74      0.76        84

    accuracy                           0.81       214
   macro avg       0.81      0.80      0.80       214
weighted avg       0.81      0.81      0.81       214

Accuracy = 0.81
Kappa = 0.60
F1 Score = 0.85
Log Loss = 6.46


In [22]:
classifier_GBM = GradientBoostingClassifier(random_state=77300)

In [23]:
n_trees = [100, 200, 300] 
learning_rates = [0.01, 0.1, 0.2] 
max_depths = [3,5,8]
subsamples = [0.8, 0.9, 1.0]

Grid_srch_parameters_list = [ [{'n_estimators':n_trees, 'learning_rate': learning_rates
                               , 'max_depth': max_depths, 'subsample': subsamples}] ] 


grid_search = GridSearchCV(estimator = classifier_GBM, param_grid = Grid_srch_parameters_list[0], scoring = score_func, cv = 5, n_jobs=-1, verbose = 2)

grid_search_GBM = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  4.5min finished


In [24]:
grid_search_GBM.best_params_
grid_search_GBM.best_estimator_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.9}

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=77300, subsample=0.9, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [25]:
pred=grid_search_GBM.predict(X_test)
predprob = grid_search_GBM.predict_proba(X_test)[:,1]

In [26]:
confusion_matrix(y_test,pred)
roc_auc_score(y_test,predprob)
print(classification_report(y_test, pred))
print("Accuracy = {:.2f}".format(accuracy_score(y_test, pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, pred, pos_label='0')))
print("Log Loss = {:.2f}".format(log_loss(y_test, pred)))

array([[113,  17],
       [ 26,  58]], dtype=int64)

0.8759157509157509

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       130
           1       0.77      0.69      0.73        84

    accuracy                           0.80       214
   macro avg       0.79      0.78      0.78       214
weighted avg       0.80      0.80      0.80       214

Accuracy = 0.80
Kappa = 0.57
F1 Score = 0.84
Log Loss = 6.94


In [27]:
classifier_SVC = SVC(probability=True,random_state=77300)

In [28]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid_search = GridSearchCV(estimator = classifier_SVC, param_grid = param_grid, scoring = score_func, cv = 5, n_jobs=-1,refit = True, verbose = 3)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   20.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=77300, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(roc_auc_score), verbose=3)

In [29]:
grid_search.best_params_
grid_search.best_estimator_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=True, random_state=77300, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
pred = grid_search.predict(X_test)
predprob = grid_search.predict_proba(X_test)[:,1]

In [31]:
confusion_matrix(y_test,pred)
roc_auc_score(y_test,predprob)
print(classification_report(y_test, pred))
print("Accuracy = {:.2f}".format(accuracy_score(y_test, pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, pred, pos_label='0')))
print("Log Loss = {:.2f}".format(log_loss(y_test, pred)))

array([[117,  13],
       [ 24,  60]], dtype=int64)

0.8867216117216118

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       130
           1       0.82      0.71      0.76        84

    accuracy                           0.83       214
   macro avg       0.83      0.81      0.81       214
weighted avg       0.83      0.83      0.82       214

Accuracy = 0.83
Kappa = 0.63
F1 Score = 0.86
Log Loss = 5.97
